In [1]:
import pandas as pd

# Вводные вопросы

## В каком формате ты предпотчёл бы хранить промежуточные данные при работе в Pandas? В чём ограничения или неудобства этого формата?

Перед ответом собеседуемому следует уточнить что это за данные и какой их характер (текстовые или нет, содержат ли они специфичные питоновские объекты).

Варианты ответов:

1. CSV. Минусы -- не подходит для текстовых данных т.к. сам текстовый. Плюсы -- открывается чем угодно, поддреживает архивацию на лету и чтение по чанкам и по колонкам в Пандас.
2. HDF. Минусы -- немного сложен в использовании, т.к. это по сути файловая система. Плюсы -- всё остальное.
3. Joblib, pickle. Минусы -- открываются только в питоне, не безопасны. Плюсы -- хранят питоновские объекты, бинарные.
4. Базы данных. Минусы -- сложно переносимы (не скинешь файлом), необходим сервер СУБД. Плюсы -- язык запросов, возможность частичной загрузки.
5. Message pack -- Deprecated since version 0.25.0.
6. Parquet. Минусы -- не поддерживает многие типы данных. Плюсы -- можно указывать, какие колонки загружать, а какие нет, поддерживает архивацию.
7. Feather. Минусы -- не поддерживает многие типы данных. Плюсы -- удобен для одновременной работы в Python и R.
8. Excel. Неудобен для больших данных.

## Что делать, если файл который необходимо считать, не помещается в память?

Некоторые форматы поддерживают поколоночное считывание или чтение по чанкам.

## Как уменьшить размер датафрейма?

Конвертировать типы. Архивировать. Разбить на колонки и хранить отдельно.

## Вопрос на умение векторизировать операции

Дано: DataFrame вида

In [106]:
df = pd.DataFrame(
    data=[[1, 2, "3"], [4, 5, "6"], [7, 8, "9"]],
    columns=list("abc"),
    index=["I", "II", "III"]
)

Запиши следующий ~~говно~~код одной строкой

Вариант 1

In [ ]:
df["d"] = 0

for index, row in df.iterrows():
    row_sum = 0
    for num, item in enumerate(row):
        if isinstance(item, int):
            row_sum += item
    df.loc[index, "d"] = row_sum

Ответ

In [ ]:
df["d"] = df.sum(axis=1)

Вариант 2

In [ ]:
for index, row in df.iterrows():
    for num, item in enumerate(row):
        df.loc[index, "b"] = "item " + str(item)

Ответ

In [107]:
# наилучший вариант

df["b"] = 'item ' + df['b'].astype(str)

In [61]:
# допустимо

df["b"] = df["b"].apply(lambda x: "item " + str(x))

## Вопрос на понимание области видимости и присваивания по ссылке

Запиши получившийся датафрейм `df3`.

In [119]:
df = pd.DataFrame(
    data=[["1", "2", "3"], [4, 5, 6], [7, 8, 9]],
    columns=list("abc"),
    index=["I", "II", "III"]
)


def multiply_first_column_by_n(dataframe: pd.DataFrame, n: int):
    """
    Просто функция, которая умножает первую колонку на число
    """
    dataframe.iloc[:, 0] = dataframe.iloc[:, 0] * n
    return dataframe


df2 = multiply_first_column_by_n(df, 2)
df3 = multiply_first_column_by_n(df, 3)

## Вопрос про индексы.

In [ ]:
# TBD maybe

## Задание на трансформацию данных. Не мега сложное, можно дать на самом собеседовании.

Большинство методов машинного обучения требуют размеченные данные, на которых они могли бы обучаться. Например, чтобы обучить классификатор определять наличие этнического конфликта в текстах социальных медиа, необходимо чтобы несколько человек проставили метки нескольким тысячам текстов. Именно с такими данными вам и предостоит сегодня работать -- это данные из статьи [Detecting Interethnic Relations with the Data from Social Media](https://link.springer.com/chapter/10.1007%2F978-3-319-69784-0_2).

In [4]:
labels = pd.read_csv("../data/labels.csv")

labels.head()

,has_eth_conflict_raw_value,document.id,assessor
0,NaN,322315268,skuchilina
1,NaN,322315268,DariaN
2,NaN,322315268,adzhigitova
3,0.0,323667074,an_men
4,0.0,323667074,DariaN


Важной проблемой в ручном кодировании является [проблема согласия](https://en.wikipedia.org/wiki/Inter-rater_reliability) кодировщиков между собой -- люди часто проставляют разные метки одним и тем же текстам. Для определения степени согласия часто используется метрика под названием [альфа Криппендорфа](https://en.wikipedia.org/wiki/Krippendorff%27s_alpha). Считается, что если она меньше 0.66, разметке кодировщиков доверять нельзя.

Для расчёта альфы Криппендорфа вначале необходимо посчитать так называемые reliability data, чем вы сейчас и займётесь. Reliability data выглядять следующим образом -- каждая колонка соответствует отдельному документу, каждая страка -- кодировщику. В ячейках, соответственно, находятся метки, проставленные кодировщиками документам. Подробнее можно посмотреть в статье на Википедии. 

Ответ

In [8]:
reliability_data = (
    labels.groupby(["document.id", "assessor"])
    .first()
    .reset_index()
    .pivot(index="assessor", columns="document.id")
)

###################
# !pip install krippendorff
import krippendorff

krippendorff.alpha(reliability_data.values, level_of_measurement="ordinal")

0.4220650186191617

## Вопрос про последовательности. Задание сложное, можно дать на дом

Дано: таблица покупками `purchases.csv`.

Колонки:

- дата покупки
- id чека
- id клиента
- id категории.

Найти: матрицу категории$\times$категории, ячейки которой содержали бы информацию о том, сколько раз товар из каждой категории был куплен в следующем чеке после каждой другой категории. **Если между покупками прошло более 30 дней, то покупки перестают считаться частью одной последовательнсти**. Иными словами, вам необходимо посчитать матрицу последовательностей покупок.

NB: таблица с покупками может систоять из сотен миллионном записей, поэтому постарайтесь написать оптимальный код.

Решение

In [2]:
import json
import numpy as np
import pandas as pd

from itertools import product

bills = pd.read_csv("../data/purchases.csv", parse_dates=["DATE_SQL"])
bills.head()

,DATE_SQL,TRANSACTION_NUMBER,BUSINESS_PARTNER,SYNT_CATALOG
0,2018-01-11,S078/4/487360,400,152
1,2018-03-19,S078/4/560068,400,108
2,2018-01-27,S240/6/581679,668,50
3,2018-01-27,S240/6/581679,668,21
4,2018-01-06,S667/4/36038,668,105


In [3]:
bills_grouped = bills.groupby(["BUSINESS_PARTNER", "TRANSACTION_NUMBER"]).agg(
    date=pd.NamedAgg(column='DATE_SQL', aggfunc='first'),
    categories=pd.NamedAgg(column="SYNT_CATALOG", aggfunc=set)
)

# Алгоритм с матричными вычислениями
bills_grouped = (
    bills_grouped
    .reset_index(level=0)
    .sort_values(by=["BUSINESS_PARTNER", "date"])
)
bills_grouped["shifted_categories"] = bills_grouped.categories.shift(1)

# Помечаем первые покупки в каждой цепочке
bills_grouped["shifted_categories"] = (
    bills_grouped
    .shifted_categories
    .mask(cond=~(bills_grouped["BUSINESS_PARTNER"].duplicated(keep="first"))))

# Помечаем, если различие между покупками в одной цепочке больше 30 дней.
bills_grouped["date_diff"] = (
    bills_grouped.date - bills_grouped.date.shift(1)).dt.days
bills_grouped["date_diff"] = (
    bills_grouped["date_diff"]
    .apply(lambda x: np.nan if x > 30 else x)
)

# Удаляем отмеченное
bills_grouped.dropna(subset=["shifted_categories", "date_diff"], inplace=True)

# Самая медленная часть, не удалось уйти от циклов. Итеративно заполняем матрицу.

# Создаём матрицу
categories = bills.SYNT_CATALOG.unique()
categories_matrix = pd.DataFrame(index=categories, columns=categories).fillna(0)

# Заполняем
for i in zip(bills_grouped.categories.tolist(), bills_grouped.shifted_categories.tolist()):
    p = product(i[0], i[1])
    for j in p:
        categories_matrix.loc[j[0], j[1]] += 1

categories_matrix

,152,108,50,21,105,129,109,161,1,77,...,194,200,181,159,59,98,188,45,78,36
152,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
108,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
105,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
188,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
78,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
